In [1]:
%matplotlib inline  
import matplotlib.pyplot as plt   
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook
from time import sleep
import scipy
import operator
import difflib
import math
from IPython.core.display import display,HTML
try:
    import cPickle as pickle     #it is faster than pickle!
except:
    import pickle
    
import unicodedata
import networkx as nx
import itertools
import seaborn as sns   ### https://seaborn.pydata.org/tutorial/categorical.html
    
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser



import plotly.plotly as py
from plotly.graph_objs import *

# i only need my credentials if i want to plot online --- and send plots to server (limits per day apply!)
#import plotly.tools as tls
#tls.set_credentials_file(username='juliettapc', api_key='deyNIvtOoDZ5PLmrHlhd')  # my plotly account credentials


import pygraphviz
from networkx.drawing.nx_agraph import graphviz_layout




########## to be able to plot offline (without sending the plots to the plotly server every time)
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [ ]:
import plotly
plotly.__version__

In [2]:

path="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Dropbox/"
input_file='Dropbox_datafile_may22_2017_modified.csv'
df=pd.read_csv(path+input_file, sep=',',na_values=["NAN","-1","null"],low_memory=False, parse_dates=['folder_creation_date','date_last_change']) # set header=0 if i wanna pass it my own list of header names
df = df.drop('Unnamed: 0', 1)
print df.shape
df.head()

(2592500, 15)


,folder_id,num_folder_members,folder_creation_date,date_last_change,user_id,email_domain,num_adds,num_edits,num_deletes,major_content_type,major_content_ext,group_total_publ,group_num_papers_last,group_num_citations,folder_lifespan
0,592732,4,2008-06-12,2012-12-25,77422,cornell.edu,NaN,NaN,NaN,office_docs,doc,NaN,NaN,NaN,1658
1,592732,4,2008-06-12,2012-12-25,79630,cornell.edu,NaN,NaN,NaN,office_docs,doc,NaN,NaN,NaN,1658
2,592732,4,2008-06-12,2012-12-25,77619,cornell.edu,80.0,72.0,0.0,office_docs,doc,NaN,NaN,NaN,1658
3,604794,2,2008-06-27,2012-12-23,86572,unc.edu,21798.0,40498.0,10660.0,pdf,pdf,NaN,NaN,NaN,1641
4,680152,2,2008-08-26,2013-01-04,76579,cs.princeton.edu,644.0,0.0,112.0,pdf,pdf,NaN,NaN,NaN,1593


In [ ]:
df.shape

In [ ]:
for item in df.email_domain.unique():
    print item,"   "

In [3]:
#### time window for temporal networks
string_start_date='2014-05-01'
start_date=pd.Timestamp(string_start_date)

#string_end_date='2014-11-01'
end_date=start_date + pd.Timedelta('180 days')




list_tupla_dates=[]
for i in range(6):
    print start_date, end_date    
    tupla_dates=[start_date,end_date]
    list_tupla_dates.append(tupla_dates)

    
    start_date += pd.Timedelta('180 days')
    end_date += pd.Timedelta('180 days')

# 2014-05-01_2014-10-28

2014-05-01 00:00:00 2014-10-28 00:00:00
2014-10-28 00:00:00 2015-04-26 00:00:00
2015-04-26 00:00:00 2015-10-23 00:00:00
2015-10-23 00:00:00 2016-04-20 00:00:00
2016-04-20 00:00:00 2016-10-17 00:00:00
2016-10-17 00:00:00 2017-04-15 00:00:00


In [ ]:

### for plotting time series directly from data
#df.plot(x='folder_creation_date',y='folder_lifespan',ls='',marker='.') 



In [4]:
list_tupla_dates

[[Timestamp('2014-05-01 00:00:00'), Timestamp('2014-10-28 00:00:00')],
 [Timestamp('2014-10-28 00:00:00'), Timestamp('2015-04-26 00:00:00')],
 [Timestamp('2015-04-26 00:00:00'), Timestamp('2015-10-23 00:00:00')],
 [Timestamp('2015-10-23 00:00:00'), Timestamp('2016-04-20 00:00:00')],
 [Timestamp('2016-04-20 00:00:00'), Timestamp('2016-10-17 00:00:00')],
 [Timestamp('2016-10-17 00:00:00'), Timestamp('2017-04-15 00:00:00')]]

In [ ]:
#print df.folder_creation_date.min(), df.folder_creation_date.max()   ###  2008-05-01 00:00:00     2017-05-16 00:00:00
#print df.date_last_change.min(), df.date_last_change.max()   ###          2012-11-07 00:00:00     2017-05-17 00:00:00


In [ ]:
df[df.user_id == 86572]

#table[table.column_name == some_value]

In [5]:


G_all=nx.Graph()

for tupla_dates in tqdm_notebook(list_tupla_dates):
    


    start_date=tupla_dates[0]
    end_date=tupla_dates[1]

    print "buiding network for time window:",str(start_date).replace("00:00:00","").strip(),"--",str( end_date).replace("00:00:00","").strip()

    
    G=nx.Graph()

    dict_folder_list_domains={}
    dict_folder_list_users={}
    dict_user_id_domain={}
    dict_user_id_edit={}
    dict_user_id_del={}
    dict_user_id_add={}
    dict_user_id_folder_lifespan={}



    print df.shape
    cont=0


#     df_selection=df[df['first_name'].notnull() & (df['nationality'] == "USA")]
#     df_selection=df[  ( (df['folder_creation_date'] >= start_date )   & (df['folder_creation_date'] <= end_date ) ) | ( (df['date_last_change'] >= start_date )   & (df['date_last_change'] <= end_date ) ) ]
    
    
    
    for row in tqdm_notebook(df.iterrows()):

        folder=row[1]["folder_id"]
        user_id=row[1]["user_id"]                              
        email=row[1]["email_domain"]    
        edits=row[1]["num_edits"]
        adds=row[1]["num_adds"]
        deletes=row[1]["num_deletes"]   
        folder_lifespan=row[1]["folder_lifespan"]
        num_members=row[1]["num_folder_members"]


        folder_creation_date=row[1]["folder_creation_date"]
        date_last_change=row[1]["date_last_change"]


        #     for focus_university in list_focus_univ:        ALL UNIVERSITIES            
    #         if focus_university in email:
    #             flag_useful_line =1
    #             break


    
        flag_useful_line=0    
        if (folder_creation_date >= start_date  and folder_creation_date <= end_date)     or     ( date_last_change >= start_date  and  date_last_change <= end_date ):
            flag_useful_line =1



        if flag_useful_line ==1:
                try: 
                    dict_folder_list_users[folder].append(user_id)
                except:
                    dict_folder_list_users[folder]=[]
                    dict_folder_list_users[folder].append(user_id)

                try:    
                    dict_folder_list_domains[folder].append(email)
                except:            
                    dict_folder_list_domains[folder]=[]
                    dict_folder_list_domains[folder].append(email)



                dict_user_id_domain[user_id]=email   

                try:
                    dict_user_id_edit[user_id] +=edits   # ojo!! the records are by folder and by user, so i need to accumulate!
                    dict_user_id_del[user_id] +=deletes
                    dict_user_id_add[user_id] +=adds 

                    dict_user_id_folder_lifespan[user_id].append(folder_lifespan) 

                except :
                    dict_user_id_edit[user_id] =edits
                    dict_user_id_del[user_id] =deletes
                    dict_user_id_add[user_id] =adds 

                    dict_user_id_folder_lifespan[user_id]=[]
                    dict_user_id_folder_lifespan[user_id].append(folder_lifespan) 



 

    print "done with the dict., size:",len(dict_folder_list_users)




    for user in  dict_user_id_folder_lifespan:
        dict_user_id_folder_lifespan[user]=np.median(dict_user_id_folder_lifespan[user])



    for folder in dict_folder_list_users:
        dict_folder_list_users[folder]=list(set(dict_folder_list_users[folder]))  # remove possible duplicates

        dict_folder_list_domains[folder]=list(set(dict_folder_list_domains[folder]))


        lista=dict_folder_list_users[folder]

        if len(lista)>1:

            lista_pares=itertools.combinations(lista, 2)        

            for item in lista_pares:
                e1=item[0]
                e2=item[1]
                G.add_edge(e1,e2)
               # print "added edge:",e1, e2
    


    print "  N:", len(G.nodes()),"  L:", len(G.edges())       
    GC = max(nx.connected_component_subgraphs(G), key=len)
    print "\n  GC:     N:", len(GC.nodes()), "  L:", len(GC.edges())





    #### the master dict is larger than G.nodes(), and the set_node_attribute does not work in that case, so i get a selection of the dict only with the common nodes:
    #######################
    print len(G.nodes()), len(dict_user_id_domain)

    new_dict_user_id = {k: dict_user_id_domain[k] for k in set(G.nodes()) & set(dict_user_id_domain.keys())}
    print len(G.nodes()), len(new_dict_user_id)
    # i add the email as node attributes    
    nx.set_node_attributes(G, 'email', new_dict_user_id)
    ############



    new_dict_user_id = {k: dict_user_id_edit[k] for k in set(G.nodes()) & set(dict_user_id_edit.keys())}
    # i add the number of edits as node attributes    
    nx.set_node_attributes(G, 'edits', new_dict_user_id)
    ############



    new_dict_user_id = {k: dict_user_id_del[k] for k in set(G.nodes()) & set(dict_user_id_del.keys())}
    # i add the number of deletes as node attributes    
    nx.set_node_attributes(G, 'deletes', new_dict_user_id)
    ############



    new_dict_user_id = {k: dict_user_id_add[k] for k in set(G.nodes()) & set(dict_user_id_add.keys())}
    # i add the number of additions as node attributes    
    nx.set_node_attributes(G, 'adds', new_dict_user_id)
    ############



    new_dict_user_id = {k: dict_user_id_folder_lifespan[k] for k in set(G.nodes()) & set(dict_user_id_folder_lifespan.keys())}
    # i add the median folder_lifespan as node attributes 
    nx.set_node_attributes(G, 'median_folder_lifespan', new_dict_user_id)
    ############




    

    

    path="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/Networks/"
    filename="first_network_"+str(start_date).replace("00:00:00","").strip()+"_"+str(end_date).replace("00:00:00","").strip()+".pickle"
    with open(path+filename,'wb') as f:
        pickle.dump(G, f)
    print "written:",path+filename


    
    ###### global network
    G_all.add_nodes_from(G.nodes(data=True))
    G_all.add_edges_from(G.edges(data=True))

    
print "\n\n"    
    



print "\n\n Global network  N:", len(G_all.nodes()),"  L:", len(G_all.edges())       
GC_all = max(nx.connected_component_subgraphs(G_all), key=len)
print "\n  GC:     N:", len(GC_all.nodes()), "  L:", len(GC_all.edges())



print "\n\n"    
    

path="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/Networks/"
filename="first_network_all.pickle"
with open(path+filename,'wb') as f:
    pickle.dump(G_all, f)
print "written:",path+filename




buiding network for time window: 2014-05-01 -- 2014-10-28
(2592500, 15)



KeyboardInterrupt: 

In [7]:

print "\n\n Global network  N:", len(G_all.nodes()),"  L:", len(G_all.edges())       
GC_all = max(nx.connected_component_subgraphs(G_all), key=len)
print "\n  GC:     N:", len(GC_all.nodes()), "  L:", len(GC_all.edges())




 Global network  N: 340563   L: 0

  GC:     N: 1   L: 0
